# 노이즈 기반 코인 자동 매수 & 매도

## 1단계 import 및 기본 세팅

In [1]:
from os import access
from re import L
import time
from numpy import absolute
import pyupbit
import datetime
import pandas as pd
from pyupbit.quotation_api import get_tickers
import requests

In [2]:
access = ""
secret = ""

upbit = pyupbit.Upbit(access, secret)

In [3]:
# slack에 메세지 봇 추가


def post_message(token, channel, text):
    response = requests.post(
        "https://slack.com/api/chat.postMessage",
        headers={"Authorization": "Bearer " + token},
        data={"channel": channel, "text": text},
    )
    print(response)


myToken = "slackbot token"

## 2단계 함수 모음

In [4]:
# 매수 목표가 조회
def get_target_price(ticker, k):
    """변동성 돌파 전략으로 매수 목표가 조회"""
    try:
        df = pyupbit.get_ohlcv(ticker, interval="day", count=2)
        target_price = df.iloc[0]["close"] + \
            (df.iloc[0]["high"] - df.iloc[0]["low"]) * k
    except:
        pass
    
    return target_price

In [5]:
# 잔고조회
def get_balance(ticker):
    """잔고 조회"""
    balances = upbit.get_balances()
    for b in balances:
        if b["currency"] == ticker:
            if b["balance"] is not None:
                return float(b["balance"])
            else:
                return 0
    return 0


In [6]:
# 현재가 조회
def get_current_price(ticker):
    """현재가 조회"""
    return pyupbit.get_orderbook(ticker=ticker)["orderbook_units"][0]["ask_price"]

In [20]:
# 3일 이동평균 조회
def get_maday3(ticker):
    """3일 이동 평균선 조회"""
    try:
        df = pyupbit.get_ohlcv(ticker, interval="day", count=3)
        maday3 = df["close"].rolling(3).mean().iloc[-1]
    except:
        maday3 = 0

    return maday3

In [8]:
# 15분 이동평균 조회
def get_mamin15(ticker):
    """15분 이동 평균선의 3회차 이동평균 조회"""
    try:
        df = pyupbit.get_ohlcv(ticker, interval="minute5", count=3)
        mamin15 = df["close"].rolling(3).mean().iloc[-1]
    except:
        pass

    return mamin15

In [9]:
# 시장 시작시간 조회
def get_start_time(ticker):
    """시작 시간 조회"""
    df = pyupbit.get_ohlcv(ticker, interval="day", count=1)
    start_time = df.index[0]
    return start_time

In [10]:
# 매도 가격 타겟팅
def get_sell_price(ticker, k):
    """15분 이평선의 하락 변동성 돌파시 매도"""
    try:
        df = pyupbit.get_ohlcv(ticker, interval="minute15", count=1)
        sell_price = get_mamin15(ticker) - (df.iloc[0]["high"] - df.iloc[0]["low"]) * k
    except:
        pass

    return sell_price

In [11]:
# 노이즈 함수
def get_noised_coin():
    """3일 노이즈 df에 추가 + 0.5아래인 값 분류"""
    tickers = pyupbit.get_tickers(fiat="KRW")
    df = pd.DataFrame()
    for ticker in tickers:
        try:
            temp = pyupbit.get_ohlcv(ticker, interval="day", count=3)
            temp["ticker"] = ticker
            df = pd.concat([df, temp])
        except:
            pass
    df_noise = pd.DataFrame()
    for ticker in tickers:
        try:
            temp = df
            temp["noise"] = 1 - (
                absolute(df["open"] - df["close"]) /
                absolute(df["high"] - df["low"])
            )
            df_noise = pd.concat([df, temp])
        except:
            pass

    noised_coin = []

    for ticker in tickers:
        check = df_noise[df_noise["ticker"] == ticker]
        if check["noise"].mean() < 0.5:
            # print(ticker, check['noise'].mean())
            noised_coin.append(ticker)
    return noised_coin

In [12]:
# 노이즈가 포함된 df 출력
def get_noised_df():
    """노이즈 0.5이하인것 조회"""
    tickers = pyupbit.get_tickers(fiat="KRW")
    df = pd.DataFrame()
    for ticker in tickers:
        try:
            temp = pyupbit.get_ohlcv(ticker, interval="day", count=3)
            temp["ticker"] = ticker
            df = pd.concat([df, temp])
        except:
            pass
    df_noise = pd.DataFrame()
    for ticker in tickers:
        try:
            temp = df
            temp["noise"] = 1 - (
                absolute(df["open"] - df["close"]) /
                absolute(df["high"] - df["low"])
            )
            df_noise = pd.concat([df, temp])
        except:
            pass
    return df_noise

In [13]:
# 매수, 매도의 target df만들기(기존 방식으로는 내가 원하는 코인을 원하는 가격에 매수 불가: 자료구조 문제)
def get_target_df(tickers):
    target_df = pd.DataFrame(
        columns=["coin", "target_price", "maday3", "sell_price", "k"]
    )
    for coin in tickers:
        df_noise = get_noised_df()
        check = df_noise[df_noise["ticker"] == coin]
        k = check["noise"].mean()
        target_price = get_target_price(coin, k)
        maday3 = get_maday3(coin)
        sell_price = get_sell_price(coin, k)
        # DataFrame에 특정 정보를 이용하여 data 채우기
        target_df = target_df.append(
            pd.DataFrame(
                [[coin, target_price, maday3, sell_price, k]],
                columns=["coin", "target_price", "maday3", "sell_price", "k"],
            ),
            ignore_index=True,
        )
    target_df.set_index("coin", inplace=True)
    return target_df

In [14]:
# 매도 가격 5분마다 업데이트
def get_updateSell_price(df, ticker, k):
    df = df
    df["sell_price"] = df["sell_price"].map(get_sell_price(ticker, k))
    return df

## 3단계 자동매수 매도 시작

In [15]:
noised_coin = None
target_df = None

# 시작 메세지 슬랙 전송
post_message(myToken, "#history", "시스템 시작")
print("Trade System Start")

<Response [200]>
Trade System Start


In [16]:
now = datetime.datetime.now()
start_time = get_start_time("KRW-BTC")
end_time = start_time + datetime.timedelta(days=1)
fee = 0.0005
current_coin = []
noised_coin = get_noised_coin()
print(noised_coin)

['KRW-LTC', 'KRW-WAVES', 'KRW-TRX', 'KRW-JST', 'KRW-NEAR']


In [17]:
target_df = get_target_df(noised_coin)
target_df.head()

UnboundLocalError: local variable 'maday3' referenced before assignment

In [18]:
tickers = noised_coin
df = pd.DataFrame()
ticker = "KRW-XLM"
try:
    temp = pyupbit.get_ohlcv(ticker, interval="day", count=3)
    temp["ticker"] = ticker
    df = pd.concat([df, temp])
except:
    pass

df_noise = pd.DataFrame()

try:
    temp = df
    temp["noise"] = 1 - (
        absolute(df["open"] - df["close"]) /
        absolute(df["high"] - df["low"])
    )
    df_noise = pd.concat([df, temp])
except:
    pass
    
df_noise.head()

,open,high,low,close,volume,value,ticker,noise
2021-12-16 09:00:00,342.0,343.0,320.0,322.0,7.119721e+07,2.371594e+10,KRW-XLM,0.130435
2021-12-17 09:00:00,322.0,327.0,309.0,319.0,6.395271e+07,2.041179e+10,KRW-XLM,0.833333
2021-12-18 09:00:00,319.0,331.0,314.0,328.0,1.759997e+07,5.696414e+09,KRW-XLM,0.470588
2021-12-16 09:00:00,342.0,343.0,320.0,322.0,7.119721e+07,2.371594e+10,KRW-XLM,0.130435
2021-12-17 09:00:00,322.0,327.0,309.0,319.0,6.395271e+07,2.041179e+10,KRW-XLM,0.833333


In [21]:
maday3 = get_maday3("KRW-STP2")
print(maday3)

404
Code not found
404
0


In [22]:
df_noise = pd.DataFrame()
check = df_noise[df_noise["ticker"] == coin]
k = check["noise"].mean()

KeyError: 'ticker'